This is the workbook to train an NLP model for summarization and keywords identification on Standards of Finance Sustainable practice 
Notes for data cleaning 
save in a word document and then remove '/n' characters, 'page numbers', 
next step explore with hugging face tokenizers


In [ ]:
#pip install pdf reader 
#pip install langchain
#pip install huggingface hub
#pip install sentence transformers
#pip install weaviate

In [26]:
pip weaviate-client

  Obtaining dependency information for sentence_transformers from https://files.pythonhosted.org/packages/06/97/57afa3d05801b6b9305f96a7ce5995e12c1d2ba25ce66747de107816b0b5/sentence_transformers-2.3.1-py3-none-any.whl.metadata
  Obtaining dependency information for transformers<5.0.0,>=4.32.0 from https://files.pythonhosted.org/packages/85/f6/c5065913119c41ecad148c34e3a861f719e16b89a522287213698da911fc/transformers-4.37.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 2.0 MB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for torch>=1.11.0 from https://files.pythonhosted.org/packages/c7/4e/578c4e3c7ac486cddcce3e85e4704a474854835baea4eba8bc707d4a0823/torch-2.2.0-cp311-none-macosx_10_9_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 17.6 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 17.4 MB/

In [11]:
#reading pdf file from local machine
from pypdf import PdfReader
climate_framework = []

reader = PdfReader("/Users/pelumioluwaabiola/Downloads/climate_framework.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[2]
text = page.extract_text()

for i in range(0, number_of_pages):
    page = reader.pages[i]
    text = page.extract_text()
    climate_framework.append(text)

print(climate_framework)

['1\nA CLIMATE DISCLOSURE \nFRAMEWORK\nFOR SMALL AND MEDIUM-SIZED  \nENTERPRISES (SME s)\nNOVEMBER 2021\nDISCL OSURE INSI GHT ACTION', '1CONTENTS\n02 OVERVIEW\n04 INTRODUCTION\n04  BACKGROUND\n04  DEFINING SME\n05  NOTE ON MICRO AND SMALL SMES \n05  PURPOSE \n05  OBJECTIVES \n05  DEVELOPMENT \n06  INTENDED USERS \n06  ALIGNMENT AND MAPPING WITH OTHER FRAMEWORKS \n07 GUIDING PRINCIPLES \n07  PURPOSE OF PRINCIPLES \n07  FRAMEWORK PRINCIPLES \n08 MODULES: REPORTING REQUIREMENTS AND RECOMMENDATIONS \n08  HOW TO USE THE FRAMEWORK \n09  MEASURE \n11  COMMIT \n13  ACTION AND IMPACT \n14  ENERGY \n15  VALUE CHAIN EMISSIONS \n16  MANAGEMENT AND RESILIENCE\n19  CLIMATE SOLUTIONS \n20  CONCLUSION AND AREAS FOR FUTURE WORK \n20  CONTRIBUTIONS \n21  APPENDIX \n22  EXAMPLES OF CLIMATE-RELATED INITIATIVE TYPES\nVersion Publication date Revisions\n1.0 25 November 2021 -\n1.1   10 December 2021  Corporate net-zero definition updated to align with the SBTi Corporate Net-Zero Standard \nImportant Notice\

In [21]:
#chunking the text into sentences
from langchain.text_splitter import RecursiveCharacterTextSplitter

textsplitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=0.25,
    length_function=len,
    is_separator_regex=False,
)

sentences = textsplitter.create_documents(climate_framework)
print(sentences)
print(f'Total number of chunks are : {len(sentences)}')
#print first chunk
print(sentences[0])
#print second chunk
print(sentences[1])

[Document(page_content='1\nA CLIMATE DISCLOSURE \nFRAMEWORK\nFOR SMALL AND MEDIUM-SIZED  \nENTERPRISES (SME s)\nNOVEMBER 2021\nDISCL OSURE INSI GHT ACTION'), Document(page_content='1CONTENTS\n02 OVERVIEW\n04 INTRODUCTION\n04  BACKGROUND\n04  DEFINING SME\n05  NOTE ON MICRO AND SMALL SMES \n05  PURPOSE \n05  OBJECTIVES \n05  DEVELOPMENT \n06  INTENDED USERS \n06  ALIGNMENT AND MAPPING WITH OTHER FRAMEWORKS \n07 GUIDING PRINCIPLES \n07  PURPOSE OF PRINCIPLES \n07  FRAMEWORK PRINCIPLES \n08 MODULES: REPORTING REQUIREMENTS AND RECOMMENDATIONS \n08  HOW TO USE THE FRAMEWORK \n09  MEASURE \n11  COMMIT \n13  ACTION AND IMPACT \n14  ENERGY \n15  VALUE CHAIN EMISSIONS \n16  MANAGEMENT AND RESILIENCE'), Document(page_content='19  CLIMATE SOLUTIONS \n20  CONCLUSION AND AREAS FOR FUTURE WORK \n20  CONTRIBUTIONS \n21  APPENDIX \n22  EXAMPLES OF CLIMATE-RELATED INITIATIVE TYPES\nVersion Publication date Revisions\n1.0 25 November 2021 -\n1.1   10 December 2021  Corporate net-zero definition updated 

In [31]:
#import and use huggingface embeddings through langchain
from langchain_community.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

sentences_emebeddings = []
for sentence in sentences:
    sentence_embedding = embeddings.embed_query(sentence.page_content)
    sentences_emebeddings.append(sentence_embedding)

print(len(sentences_emebeddings))

[[-0.03129096329212189, 0.0002661997568793595, -0.02158201113343239, 0.04093501716852188, -0.005807432811707258, -0.03226979449391365, 0.03512857109308243, 0.028429362922906876, 0.10146445780992508, -0.0031429510563611984, 0.03461846336722374, 0.08906497061252594, 0.03543192520737648, 0.0637378841638565, 0.059639234095811844, 0.0008775524329394102, 0.024208547547459602, 0.015757227316498756, -0.038198839873075485, -0.01855364628136158, 0.023945285007357597, -0.01003908272832632, 0.03840584680438042, -0.022817743942141533, 0.047124702483415604, -0.02313932403922081, 0.006641772110015154, -0.02076037786900997, -0.014404288493096828, 0.022776415571570396, 0.016865380108356476, -0.011082552373409271, 0.04544650763273239, -0.032787833362817764, 1.8713819827098632e-06, -0.04446049779653549, -0.06926220655441284, -0.03440692648291588, 0.03170008212327957, 0.05712113156914711, 0.028052281588315964, -0.04401228949427605, -0.012408079579472542, 0.007740090135484934, 0.021204495802521706, -0.0296

In [ ]:
#store the embeddings in a vectorbase
